
# ЛАБОРАТОРНА РОБОТА  

## "Лінійні та нелінійні регресійні моделі МН"

__Метою__ лабораторної роботи є набуття практичних навичок використання модулів бібліотеки `Scikit-learn` для вирішення наступних задач:

- визначення суттєвих показчиків для регресійної моделі
- пошук та настроювання гіперпараметрів лінійних та нелінійних регресійних моделей

__Результатом__ виконання лабораторної роботи є серія моделей які прогнозують ціну кватрир на вторинному ринку житла

In [13]:
import pandas as pd
import numpy as np

data = pd.read_csv("apartment_transformed.csv")
data.shape

(398, 176)

In [14]:
# відокремити ціловий показчик 'price'
price = data["Price"]

# зилишити в 'data' тільки незалежні показчики
data = data.drop(columns = ["Price"])

data.shape, price.shape    # ((676, 301), (676,))

((398, 175), (398,))

## 1. Пошук значущих ознак

### Теоретичне введення

__Значущі (суттеві) ознаки__ це дані, які мають сильну кореляцію або вплив на результат або прогноз моделі.

Ці позазчики визначаються за допомогою процесу, який називається __відбором ознак__ ([Feature selection](https://en.wikipedia.org/wiki/Feature_selection)), який передбачає _оцінку_ та _ранжування важливості_ різних змінних у наборі даних. Це можна зробити за дпомогою статистичних тестів, кореляційного аналізу або алгоритмів машинного навчання.

Після визначення значущих ознак їх можна використовувати для навчання моделі машинного навчання.

Це може бути особливо важливим у моделях, де дані складні та містять багато змінних. Тому ідентифікація  суттєвих ознак може допомогти зменшити розмірність даних і покращити продуктивність моделі.

Класи в модулі [sklearn.feature_selection](https://scikit-learn.org/stable/modules/feature_selection.html) можна використовувати для вибору функцій/зменшення розмірності на вибіркових наборах або для покращення показників точності оцінювачів, або для підвищення їх продуктивності на масивах даних з дуже великою розмірністю.

### Завдання

Відібрати з вхідного набору `data` 7 найбільш суттєвих показчиків для регрісійної моделі машинного навчання.

In [15]:
# імпортувати з модуля 'feature_selection' селектор ознак 'SelectKBest' 
# та регрісійний тест 'f_regression'
from sklearn.feature_selection import SelectKBest, f_regression

In [16]:
# побудувати селектор 7 ознак на f-регресорі
kbest_selector = SelectKBest(f_regression, k = 7)

# застосувати селектор для побудови списку ознак
data_selected = kbest_selector.fit_transform(data, price)

In [17]:
# зберегти імена визначених селектором найбільш суттєвих ознак
best_features = kbest_selector.get_feature_names_out()

In [18]:
# побудувати датафрейм на визначених ознаках
data = pd.DataFrame(data_selected, columns = best_features)
data.head()

,ohe_categirical_encoder__street_Мічуріна,ohe_categirical_encoder__street_Саксаганського,numeric_encoder__rooms,numeric_encoder__price_per_m2,numeric_encoder__area_total,numeric_encoder__area_living,numeric_encoder__area_comfort
0,0.0,0.0,-1.125471,-0.139455,-1.048667,-1.011456,-0.424751
1,0.0,0.0,-0.160089,-0.117124,-0.287044,-0.148058,-0.315639
2,0.0,0.0,-0.160089,-0.015546,-0.265888,-0.579757,0.011697
3,0.0,0.0,-0.160089,-0.118144,-0.244731,-0.148058,-0.388380
4,0.0,0.0,-1.125471,-0.105013,-0.985198,-0.148058,-0.788458


### Висновки

_описати загальну статистичну характеристику отриманого датасети та зробити висновки щодо можливості його використання для подальшого аналізу_

Було проведено аналіз загальних статистичних характеристик датасету та зроблено висновки щодо його придатності для подальшого використання в аналітичних дослідженнях.

## 2. Множинна лінійна регресія

### Теоретичне введення

__Множинна лінійна регресія__ — це статистичний метод, який використовується для встановлення зв’язку між _залежною_ (цільвою) змінною $\textbf y$ та _кількома_ незалежними змінними $\textbf [X]$.  

__Метою__ множинної лінійної регресії є знаходження найкращого лінійного зв’язку між залежною змінною та незалежними змінними, який виражається у вигляді рівняння:
$$y = b_0 + b_1 x_1 + b_2 x_2 + ... + b_n x_n$$

Найкращій зв'язок забезпечується знаходженням таких коєфіцієнтів $[B]$, що додають мінімум обраній метриці (MSE, MAE, ...)

### Завдання


Порахувати показчики якості моделі [лінйной множинної регресії](https://uk.mcfairbanks.com/719-multiple-regression-formula) на визначениx п.1 значущих ознаках датасету застосувавши [кросс-валідацію з __10__ сплітами](https://scikit-learn.org/stable/modules/cross_validation.html).

In [19]:
# імпортувати та побудувати лінійний регресор з параметрами за замовчанням
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

In [20]:
# імпортувати крос-валідатор 'cross_validate' з модуля 'model_selection'
from sklearn.model_selection import cross_validate

In [21]:
# отримати результати крос-валідації по параметрам 'neg_mean_absolute_percentage_error' 
# та 'r2' на 10 сплітах передбачивши розрахунок на навчальному наборі 'return_train_score'
cv_results_mul = cross_validate(lr, data, price, cv = 10,
                         scoring = ("neg_mean_absolute_percentage_error","r2"),
                         return_train_score = True)

#### занести результати в датафрейм 'cv_results_mul ' наступного вигляду:

------------

|помилка тесту в %   |коєф. R2 тесту  | помилка навчання в %  | коєф. R2 навчання  |
| :------------:|:------------:|:------------:|:------------:|
|  xx.xx | xx.xx  | xx.xx  | xx.xx  |
|  xx.xx | xx.xx  | xx.xx  | xx.xx  |
|  ... | ...  | ...  | ...  |


In [24]:
cv_results_mul = pd.DataFrame({'помилка тесту в %': cv_results_mul['test_neg_mean_absolute_percentage_error'],
                              'коєф. R2 тесту': cv_results_mul['test_r2'],
                              'помилка навчання в %': cv_results_mul['train_neg_mean_absolute_percentage_error'],
                              'коєф. R2 навчання': cv_results_mul['train_r2']})
cv_results_mul.head()

,помилка тесту в %,коєф. R2 тесту,помилка навчання в %,коєф. R2 навчання
0,-0.391580,0.980588,-0.475435,0.866733
1,-0.526764,0.631274,-0.437881,0.985897
2,-0.385269,0.866260,-0.426024,0.986208
3,-0.557975,0.762530,-0.434469,0.986042
4,-0.332169,0.862515,-0.452417,0.985995


In [25]:
# продовжити наступні команди виводу:
print ("середня помилка навчання = " , np.mean(cv_results_mul['помилка навчання в %']))
print ("середня помилка тесту = ", np.mean(cv_results_mul['помилка навчання в %']))

середня помилка навчання =  -0.4352182231336667
середня помилка тесту =  -0.4352182231336667


### Висновки

_зпираючись на отримані метрики якості зробити висновок про придатність моделі, недонавчана чи перенавчана вона і т.п._
Значення помилки для навчальної та тестової вибірок є досить низькими, що свідчить про хорошу роботу моделі. 

## 3. Гребнева (Ridge) регресія

### Теоретичне введення

__Гребнева регресія__ — це техніка регулярізації, яка використовується в машинному навчанні для запобігання перенавчанню лінійних регресійних моделей за рахунок додавання штрафу до функції втрат регресійної моделі, яка зменшує величину коефіцієнтів до нуля.

$$\min_w\sum_{i=1}^n(y_i - x_i^w)^2 + \lambda|w|_2^2$$

Розмір штрафу визначається [нормою L2](https://craftappmobile.com/l1-vs-l2-regularization/) вектора коефіцієнтів, помноженою на гіперпараметр $\large \lambda$.




### Завдання

Побудувати модель на основі `ridge-регресії` та за допомогою [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) знайти таке значення _L2-регулярізатора_, яке буде
мінізувати обрані метрики якості моделі.
Для побудови моделі скоистатися датасетом, що отримано в  __завданні 1__ лабораторної роботи


In [26]:
# імпортувати ridge-регресор з модуля `sklearn.linear_model`
from sklearn.linear_model import Ridge

# побудувати регресор
ridge = Ridge(random_state=2, fit_intercept=False)

In [27]:
# імпортувати сітку пошуку `GridSearchCV` з модулю sklearn.model_selection
from sklearn.model_selection import GridSearchCV
# визначити параметр равномірного пошуку 100 значень параметеру `alpha` в диапазоні 0-100000 
grid_params = {"alpha": [0, 100000],"max_iter": [0, 100]}

In [28]:
%%time

# побудувати та натренувати гребневу регресійну модель на сітці 'grid_params'
# в якості критерія оцінки якості взяти метрику `neg_mean_absolute_percentage_error`

# створюємо сітку пошуку та тренуємо на ній модель
grid_search_model = GridSearchCV(ridge, param_grid = grid_params,
                                 scoring='neg_mean_absolute_percentage_error')

grid_search_model.fit(data, price)

CPU times: total: 359 ms
Wall time: 368 ms


C:\Users\hp\anaconda3\envs\ML\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
10 fits failed out of a total of 20.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\hp\anaconda3\envs\ML\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\hp\anaconda3\envs\ML\lib\site-packages\sklearn\linear_model\_ridge.py", line 1123, in fit
    self._validate_params()
  File "C:\Users\hp\anaconda3\envs\ML\lib\site-packages\sklearn\base.py", line 570, in _validate_params
    validate_parameter_constraints(
  File "C:\Users\hp\anaco

GridSearchCV(estimator=Ridge(fit_intercept=False, random_state=2),
             param_grid={'alpha': [0, 100000], 'max_iter': [0, 100]},
             scoring='neg_mean_absolute_percentage_error')

In [29]:
# вивести найкращій естіматор (best_estimator_), та найкраще значення обраної метрики (best_score_)
grid_search_model.best_estimator_, grid_search_model.best_score_

(Ridge(alpha=100000, fit_intercept=False, max_iter=100, random_state=2),
 -1.0106688690630032)

### Висновки

_cпираючись на отримані метрики якості зробити висновок про придатність моделі, недонавчана чи перенавчана вона і т.п._
Помилка в цій моделі більша, ніж у попередньої

## 3. Поліноміальна регресія

### Теоретичне введення

__Поліноміальна регресія__ — це тип нелінійної регресії, у якому зв’язок між незалежною змінною $\large x$ і залежною змінною $\large y$ моделюється як поліноміальна функція n-го ступеня:

$$ y = \beta_0 + \beta_1 x + \beta_2 x^2 + \beta_3 x^3 + \cdots + \beta_n x^n + \varepsilon $$

__Метою__ поліноміальної регресії є знаходження значень коефіцієнтів $ β_i$, які найкраще відповідають даним.

### Завдання


Порахувати показчики якості моделі на [поліноміальній регресії](https://uk.wikipedia.org/wiki/Поліноміальна_регресія) на визначених в п.1 значущих ознаках датасету, попередньо розширивши датасет за допомогою трансформера [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html)

In [34]:
# імпортувати модуль preprocessing.PolynomialFeatures
from sklearn.preprocessing import PolynomialFeatures

# побудувати трансформер ступеня 2 для побудови додаткових ознак в датасеті
poly = PolynomialFeatures(include_bias = False)
data_poly = poly.fit_transform(data)

In [35]:
# визначити імена відібраних показчиків
poly_features_names = poly.get_feature_names_out()

In [36]:
# побудувати датасет на визначеному поліномі `poly`
data_poly = pd.DataFrame(data_poly, columns = poly_features_names)
data_poly

,ohe_categirical_encoder__street_Мічуріна,ohe_categirical_encoder__street_Саксаганського,numeric_encoder__rooms,numeric_encoder__price_per_m2,numeric_encoder__area_total,numeric_encoder__area_living,numeric_encoder__area_comfort,ohe_categirical_encoder__street_Мічуріна^2,ohe_categirical_encoder__street_Мічуріна ohe_categirical_encoder__street_Саксаганського,ohe_categirical_encoder__street_Мічуріна numeric_encoder__rooms,...,numeric_encoder__price_per_m2^2,numeric_encoder__price_per_m2 numeric_encoder__area_total,numeric_encoder__price_per_m2 numeric_encoder__area_living,numeric_encoder__price_per_m2 numeric_encoder__area_comfort,numeric_encoder__area_total^2,numeric_encoder__area_total numeric_encoder__area_living,numeric_encoder__area_total numeric_encoder__area_comfort,numeric_encoder__area_living^2,numeric_encoder__area_living numeric_encoder__area_comfort,numeric_encoder__area_comfort^2
0,0.0,0.0,-1.125471,-0.139455,-1.048667,-1.011456,-0.424751,0.0,0.0,-0.0,...,0.019448,0.146242,0.141053,0.059234,1.099702,1.060681,0.445422,1.023044,0.429617,0.180414
1,0.0,0.0,-0.160089,-0.117124,-0.287044,-0.148058,-0.315639,0.0,0.0,-0.0,...,0.013718,0.033620,0.017341,0.036969,0.082394,0.042499,0.090602,0.021921,0.046733,0.099628
2,0.0,0.0,-0.160089,-0.015546,-0.265888,-0.579757,0.011697,0.0,0.0,-0.0,...,0.000242,0.004133,0.009013,-0.000182,0.070696,0.154150,-0.003110,0.336118,-0.006781,0.000137
3,0.0,0.0,-0.160089,-0.118144,-0.244731,-0.148058,-0.388380,0.0,0.0,-0.0,...,0.013958,0.028914,0.017492,0.045885,0.059893,0.036234,0.095049,0.021921,0.057503,0.150839
4,0.0,0.0,-1.125471,-0.105013,-0.985198,-0.148058,-0.788458,0.0,0.0,-0.0,...,0.011028,0.103459,0.015548,0.082799,0.970615,0.145866,0.776787,0.021921,0.116737,0.621666
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,0.0,0.0,-0.160089,-0.153237,-0.731324,-0.435857,-0.788458,0.0,0.0,-0.0,...,0.023482,0.112066,0.066790,0.120821,0.534835,0.318753,0.576618,0.189972,0.343655,0.621666
394,0.0,0.0,-1.125471,0.052981,-0.710168,-0.975481,-0.461122,0.0,0.0,-0.0,...,0.002807,-0.037625,-0.051682,-0.024431,0.504338,0.692755,0.327474,0.951564,0.449816,0.212633
395,0.0,0.0,-1.125471,-0.134223,-0.921730,-0.831582,-0.497492,0.0,0.0,-0.0,...,0.018016,0.123718,0.111618,0.066775,0.849585,0.766493,0.458554,0.691528,0.413706,0.247499
396,0.0,0.0,0.805294,-0.135755,-0.181263,-0.148058,-0.133786,0.0,0.0,0.0,...,0.018429,0.024607,0.020100,0.018162,0.032856,0.026837,0.024250,0.021921,0.019808,0.017899


In [37]:
# отримати результати крос-валідації на множинном регресорі `lr` по параметрам 'neg_mean_absolute_percentage_error' 
# та 'r2' на 10 сплітах передбачивши розрахунок на навчальному наборі 'return_train_score'
cv_results_poly = cross_validate(lr, data, price, cv = 10,
                         scoring = ('r2', 'neg_mean_absolute_percentage_error'),
                         return_train_score = True)

In [38]:
# занести результати крос-валідації: помилка тесту, помилка навчання та відповідні коефіцієнти 
# детермінаційї в датафрейм `cv_results_poly`. 
cv_results_poly = pd.DataFrame({'помилка тесту в %': cv_results_poly['test_neg_mean_absolute_percentage_error'],
                              'коєф. R2 тесту': cv_results_poly['test_r2'],
                              'помилка навчання в %': cv_results_poly['train_neg_mean_absolute_percentage_error'],
                              'коєф. R2 навчання': cv_results_poly['train_r2']})

cv_results_poly.head()

,помилка тесту в %,коєф. R2 тесту,помилка навчання в %,коєф. R2 навчання
0,-0.391580,0.980588,-0.475435,0.866733
1,-0.526764,0.631274,-0.437881,0.985897
2,-0.385269,0.866260,-0.426024,0.986208
3,-0.557975,0.762530,-0.434469,0.986042
4,-0.332169,0.862515,-0.452417,0.985995


In [39]:
# за допомогою крос-валідатора 'cross_val_predict' побудувати прогноз 'price_pred' 
# на лінійному регресорі на 10 сплітах
from sklearn.model_selection import cross_val_predict

price_pred = cross_val_predict(lr, data_poly, price, cv=10)

In [40]:
# вивести порівняльну таблицю з двох колонок: ціна реальна, ціна прогнозна
pred = pd.DataFrame({'ціна реальна': price,
                     'ціна прогнозна': np.round(price_pred)})
pred.head(5)

,ціна реальна,ціна прогнозна
0,30970.0,26005.0
1,82000.0,85817.0
2,135000.0,145004.0
3,84000.0,88670.0
4,42654.8,40388.0


In [42]:
# натренувати регресор `lr` на поліноміальних ознаках `data_poly`
lr.fit(data_poly, price)

LinearRegression()

In [43]:
# сформувати таблицю коєфіцієнтів поліному
coef = pd.DataFrame({'Ознаки': poly_features_names,
                     'коеф.регресора': lr.coef_.astype('int')})

### Висновки

_Базуючись на значенях метрик абсолютної помилки та r2-оцінки, сформулювати вашу думку чи відповідає поліноміальна модель вимогам якості та дати характеристику декільком коефіцієнтам (3-4) на свій вибір._ 

Модель з поліноміальною регресією має кращі показники порівняно з іншими двома моделями. Абсолютна помилка значно зменшилась порівняно з простою лінійною регресією, тоді як значення r2 збільшилось.

## 5. Зберігання побудованх моделей

In [44]:
# зберегти лінийну, гребневу та поліноміальну моделі у відпрвідних pickle-файлах:
# 'lin_model.pkl', 'ridge_model.pkl', 'poly_model.pkl'
import pickle

model_names = ['lin_model.pkl', 'ridge_model.pkl', 'poly_model.pkl']
models = [lr, ridge, poly]

for i in range(len(model_names)):
    with open(model_names[i], 'wb') as f:
        pickle.dump(models[i], f)